In [42]:
import pandas as pd

In [43]:
import xml.etree.ElementTree as et

In [44]:
import os

In [45]:
import matplotlib.pyplot as plt

In [46]:
def transform_xml(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('vehicle'):
        dict = attr.copy()
        dict.update(xml.attrib)
        
        yield dict


In [47]:
def transform_xml_tripinfo(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('tripinfo'):
        dict = attr.copy()
        dict.update(xml.attrib)
        
        yield dict

In [48]:
def transform_xml_stops(xml_doc):
    for route in xml_doc.iter('route'):
        route_dict = route.attrib.copy()
        stops = []
        
        for stop in route.findall('stop'):
            stop_dict = stop.attrib.copy()
            stops.append(stop_dict)
        
        route_dict['stops'] = stops
        
        yield route_dict

In [49]:
base_folder = "C:\\Users\\Admin\\Sumo\\normafa_b"

In [50]:
file_path = os.path.join(base_folder, "Battery.out.xml")

battery_output = et.parse(file_path)
battery_output_root = battery_output.getroot()

transform = transform_xml(battery_output_root)
battery_output_list = list(transform)

In [51]:
file_path = os.path.join(base_folder, "tripinfo.xml")

b_tripinfo_output = et.parse(file_path)
b_tripinfo_output_root = b_tripinfo_output.getroot()

transform = transform_xml_tripinfo(b_tripinfo_output_root)
b_tripinfo_output_list = list(transform)

In [52]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
stops = et.parse(file_path)

transform = transform_xml_stops(stops.getroot())
stops_list = list(transform)

file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
vehicles = et.parse(file_path)

transform = transform_xml(vehicles.getroot())
vehicles_list = list(transform)

In [53]:
battery_output_pd = pd.DataFrame(battery_output_list)
battery_output_pd.shape

(4322, 18)

In [54]:
stops_pd = pd.DataFrame(stops_list)
#b_tripinfo_output_pd.shape
#stops_pd = stops_pd.drop(stops_pd.columns[0], axis=1)
#stops_pd

vehicles_pd = pd.DataFrame(vehicles_list)
vehicles_pd = vehicles_pd.drop(vehicles_pd.columns[0], axis=1)
vehicles_pd

,id,route,type,depart,line
0,C7810510.0,C7810510,bus,21770,C7810510
1,C78105100.0,C7810510,bus,37970,C7810510
2,C78105101.0,C78105101,bus,30950,C78105101
3,C78105104.0,C7810510,bus,38510,C7810510
4,C78105105.0,C78105101,bus,31370,C78105101
...,...,...,...,...,...
9381,C77670960.0,C776701014,tram,38630,C776701014
9382,C77670970.0,C776701006,tram,37070,C776701006
9383,C77670978.0,C776701014,tram,39530,C776701014
9384,C77670988.0,C776701006,tram,37970,C776701006


In [55]:
b_tripinfo_output_pd = pd.DataFrame(b_tripinfo_output_list)
#b_tripinfo_output_pd.shape
b_tripinfo_output_pd = b_tripinfo_output_pd.drop(b_tripinfo_output_pd.columns[0], axis=1)
b_tripinfo_output_pd

,id,depart,departLane,departPos,departSpeed,departDelay,arrival,arrivalLane,arrivalPos,arrivalSpeed,...,routeLength,waitingTime,waitingCount,stopTime,timeLoss,rerouteNo,devices,vType,speedFactor,vaporized
0,C74509135.0,24890.00,1176083116_0,12.10,0.00,0.00,26200.00,126765651#2_0,52.45,3.91,...,6896.22,199.00,14,257.00,514.57,0,tripinfo_C74509135.0 battery_C74509135.0,bus,1.00,
1,C74509106.0,26330.00,126765648#0_0,12.10,0.00,0.00,27836.00,879958382_0,47.24,6.05,...,6841.14,204.00,12,511.00,473.77,0,tripinfo_C74509106.0 battery_C74509106.0,bus,1.00,
2,C74509124.0,27170.00,126765648#0_0,12.10,0.00,0.00,28676.00,879958382_0,47.24,5.74,...,6841.14,235.00,13,449.00,536.21,0,tripinfo_C74509124.0 battery_C74509124.0,bus,1.00,


In [56]:
battery_output_pd = battery_output_pd.drop(battery_output_pd.columns[0], axis=1)
battery_output_pd

,id,energyConsumed,totalEnergyConsumed,totalEnergyRegenerated,actualBatteryCapacity,maximumBatteryCapacity,chargingStationId,energyCharged,energyChargedInTransit,energyChargedStopped,speed,acceleration,x,y,lane,posOnLane,timeStopped
0,C74509135.0,0.0000,0.0000,0.0000,17500.0000,35000.0000,NULL,0.0000,0.0000,0.0000,0.0000,0.0000,3718.8992,3512.8499,1176083116_0,12.1000,0
1,C74509135.0,-0.0054,0.0000,0.0054,17500.0054,35000.0000,NULL,0.0000,0.0000,0.0000,0.6627,0.6627,3719.1436,3513.4650,1176083116_0,12.7627,0
2,C74509135.0,-0.1181,0.0000,0.1235,17500.1235,35000.0000,NULL,0.0000,0.0000,0.0000,0.2580,-0.4047,3719.2388,3513.7045,1176083116_0,13.0206,0
3,C74509135.0,-0.0121,0.0000,0.1356,17500.1356,35000.0000,NULL,0.0000,0.0000,0.0000,0.1273,-0.1307,3719.2858,3513.8227,1176083116_0,13.1480,0
4,C74509135.0,0.0237,0.0237,0.1356,17500.1119,35000.0000,NULL,0.0000,0.0000,0.0000,0.0019,-0.1254,3719.2864,3513.8244,1176083116_0,13.1498,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,C74509124.0,0.2868,3232.8149,1028.5528,15295.7379,35000.0000,NULL,0.0000,0.0000,0.0000,1.5648,0.8524,3733.7987,3445.4702,879958382_0,27.0752,0
4318,C74509124.0,0.3931,3233.2081,1028.5528,15295.3447,35000.0000,NULL,0.0000,0.0000,0.0000,2.2578,0.6930,3733.4526,3443.2401,879958382_0,29.3330,0
4319,C74509124.0,1.1873,3234.3954,1028.5528,15294.1574,35000.0000,NULL,0.0000,0.0000,0.0000,3.3122,1.0544,3732.7209,3440.0230,879958382_0,32.6452,0
4320,C74509124.0,1.0824,3235.4778,1028.5528,15293.0750,35000.0000,NULL,0.0000,0.0000,0.0000,4.0054,0.6933,3731.4362,3436.2295,879958382_0,36.6506,0


In [57]:
grouped_df = battery_output_pd.groupby('id')

list_of_dfs = [group_data for _, group_data in grouped_df]
#C74509106 = list_of_dfs[0]
#C74509124 = list_of_dfs[1]
#C74509135 = list_of_dfs[2]

In [58]:

results = []
for group_id, group_data in grouped_df:
    avg_speed = group_data['speed'].astype(float).mean()
    
    energy = float(group_data['totalEnergyConsumed'].iloc[-1])-float(group_data['totalEnergyRegenerated'].iloc[-1])
    
    time_loss = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'timeLoss'].values[0]
    route_length = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'routeLength'].values[0]
    
    route = vehicles_pd.loc[vehicles_pd['id'] == group_id, 'route'].values[0]
    count_stops = stops_pd[stops_pd['id'] == route]['stops'].apply(len).sum()
    
    
    # Store the results in a dictionary
    group_result = {
        'id': group_id,
        'avgSpeed': avg_speed,
        'battery': energy,
        'timeloss': time_loss,
        'routeLength': route_length,
        'numOfStops': count_stops
    }
    
    # Append the dictionary to the results list
    results.append(group_result)

# Convert the results list to a DataFrame
result_df = pd.DataFrame(results)
print(result_df)


            id  avgSpeed    battery timeloss routeLength  numOfStops
0  C74509106.0  4.540451  2213.6631   473.77     6841.14          19
1  C74509124.0  4.538786  2208.9196   536.21     6841.14          19
2  C74509135.0  5.261245  -639.8228   514.57     6896.22          18


In [59]:
#avg_seed_b = C74509135['speed'].astype(float).mean()
#avg_seed_b

In [60]:
#float(C74509106['totalEnergyConsumed'].iloc[-1])-float(C74509106['totalEnergyRegenerated'].iloc[-1])
#24 2214.5173999999997  2219.5933
#06 2218.0583           2222.9716
#35 -615.606            -624.5073000000002

In [61]:
#plt.plot(C74509135['energyConsumed'].astype(float).cumsum())
#plt.xlabel('Time')
#plt.ylabel('Total Energy Regenerated')
#plt.title('Total Energy Regenerated Over Time')
#plt.xticks(rotation=45)
#plt.tight_layout()
#plt.show()

# CSV

In [62]:
tableBattery = pd.read_csv('batteryData.csv', delimiter=';')

tableBattery.shape

(0, 11)

## settings

In [63]:
locSetting = 'xd'
seedSetting = 'dd'
trafficScaleSetting = 'wh'

#routeLengthSetting = :)
#numOfStopsSetting = :)
#tripIdSetting = :)
#avgSpeedSetting = :)
#timelossSetting = :)
#eleupSetting = 
#eledownSetting = 
#emissionSetting = :)

In [64]:
for index, row in result_df.iterrows():
    row_data = {
        'loc': locSetting,
        'tripId': row['id'],
        'seed': seedSetting,
        'avgSpeed': row['avgSpeed'],
        'timeloss': row['timeloss'],
        'route_length': row['routeLength'],
        'elevation_up': 0,
        'elevation_down': 0,
        'trafficScale': trafficScaleSetting,
        'numOfStops': row['numOfStops'],
        'emission': row['battery']
    }
    temp_df = pd.DataFrame([row_data])
    #any empty or all-NA columns in tableBattery are excluded before concatenating the DataFrames
    tableBattery = tableBattery.dropna(axis=1, how='all')

    tableBattery = pd.concat([tableBattery, temp_df], ignore_index=True)


'id': group_id,
        'avgSpeed': avg_speed,
        'batteryCapacity': energy,
        'timeloss': time_loss,
        'routeLength': route_length,
        'stops': count_stops

In [65]:
tableBattery
#tableBattery.to_csv('batteryData.csv', index=False, sep=';')

,loc,tripId,seed,avgSpeed,timeloss,route_length,elevation_up,elevation_down,trafficScale,numOfStops,emission
0,xd,C74509106.0,dd,4.540451,473.77,6841.14,0,0,wh,19,2213.6631
1,xd,C74509124.0,dd,4.538786,536.21,6841.14,0,0,wh,19,2208.9196
2,xd,C74509135.0,dd,5.261245,514.57,6896.22,0,0,wh,18,-639.8228
